In [17]:
import os
import xml.etree.ElementTree as ET

file_append = []

for subdir, dirs, files in os.walk(r'../Train-corpus'):
    for filename in files:
        filepath = subdir + os.sep + filename
        mytree = ET.parse(filepath)
        root = mytree.getroot()

        # Handled mw tags
        for element in root.iter('mw'):
            mw_words = ""

        # strip the text field
            for words in element.iter('w'):
                mw_words += words.text
            tag = element.attrib.get('c5')
            file_append.append(mw_words.strip() + "_" + tag)

        # Handled w tags
        for element in root.iter('w'):
            word = element.text
            tag = element.attrib.get('c5')
            tag_list = tag.split('-')
            for tags in tag_list:
                file_append.append(word.strip() + "_" + tags)

        # Handled c tags
        for element in root.iter('c'):
            file_append.append(element.text.strip() + "_" + element.attrib.get('c5'))

# mw tags i.e. multiword there are tags init
# c tags i.e. puctuations
# if tag1-tag2 then word_tag1, word_tag2
# Handle the spacing 
# Don't take words from hw tags

In [18]:
myDict = {}

for word in file_append:
    if (word in myDict):
        myDict[word] += 1
    else:
        myDict[word] = 1


In [19]:
import operator

word_dict = {}
tag_dict = {}

for element in file_append:
    temp = element.split("_")
    word = temp[0].strip()
    tag = temp[1]

    if (word in word_dict):
        word_dict[word] += 1
    else: 
        word_dict[word] = 1

    if (tag in tag_dict):
        tag_dict[tag] += 1
    else: 
        tag_dict[tag] = 1

word_dict_ordered = dict( sorted(word_dict.items(), key=operator.itemgetter(1),reverse=True))
tag_dict_ordered = dict( sorted(tag_dict.items(), key=operator.itemgetter(1),reverse=True))

In [20]:
probability_of_word_given_tag = {}

# P(Word|Tag) = P(Word_Tag)/P(Tag) = Count(Word_Tag)/Count(Tag)

for word in word_dict.keys():
    for tag in tag_dict.keys():
        temp = word+"_"+tag
        temp1 = word+"|"+tag
        if (temp in myDict):
            probability_of_word_given_tag[temp1] = myDict[temp] / tag_dict[tag]
        else:
            probability_of_word_given_tag[temp1] = 0

In [21]:
def predict( word ):
    
    if word not in word_dict:
        return "UNC"
    mx_p = -1
    tag_gen = ""
    for tags in tag_dict.keys():
        word_given_tag = word + "|" + tags
        P = 0
        if word in word_dict:
            P = (float(probability_of_word_given_tag[word_given_tag]) * int(
                tag_dict[tags])) / int(word_dict[word])
        if P > mx_p:
            mx_p = P
            tag_gen = tags
    return tag_gen


In [22]:
import glob
test_files = glob.glob("../Test-corpus/*/*.xml")
print(test_files[0])

../Test-corpus/AS/ASK.xml


In [23]:
ref = []
tagged = []

correct = 0
total = 0
mw_correct = 0
mw_total = 0
p_correct = 0
p_total = 0

for test_file in test_files:
    mytree1 = ET.parse(test_file)
    root1 = mytree1.getroot()

    """Normal words tag"""
    for element in root1.iter('w'):
        word = element.text.strip()
        tag = element.attrib.get('c5')

        tag_gen = predict(word)
        if tag == tag_gen:
            correct += 1

        if tag.find('-') != -1:
            if tag.split('-')[0] == tag_gen or tag.split('-')[1] == tag_gen:
                correct += 1
                ref.append(tag_gen)
            else:
                ref.append(tag)
        else:
            ref.append(tag)
        total += 1
        tagged.append(tag_gen)


    """Multi-words"""
    for element in root1.iter("mw"):
        word = ""
        for word_mw in element.iter('w'):
            word += word_mw.text
        if word[-1] == " ":
            word = word[0:-1]
        tag = element.attrib.get('c5')
        tag_gen = predict(word)
        if tag == tag_gen:
            mw_correct += 1

        if tag.find('-') != -1:
            if tag.split('-')[0] == tag_gen or tag.split('-')[1] == tag_gen:
                mw_correct += 1
                ref.append(tag_gen)
            else:
                ref.append(tag)
        else:
            ref.append(tag)
        mw_total += 1
        tagged.append(tag_gen)

    """Punctuations"""
    for element in root1.iter("c"):
        try:
            word = element.text.strip()
            tag = element.attrib.get('c5')
            tag_gen = predict(word)
            if tag == tag_gen:
                p_correct += 1

            if tag.find('-') != -1:
                if tag.split('-')[0] == tag_gen or tag.split('-')[1] == tag_gen:
                    p_correct += 1
                    ref.append(tag_gen)
                else:
                    ref.append(tag)
            else:
                ref.append(tag)
            p_total += 1
            tagged.append(tag_gen)
        except:
            print("")

print(correct,total)
print(correct/total)

# Correct words: 3818421
# Total words: 4159247
# 0.9180558403961102


3289953 3630025
0.9063168986439487


In [27]:
from nltk.metrics import ConfusionMatrix
import sys 

stdoutOrigin=sys.stdout 
sys.stdout = open("log.txt", "w")
cm = ConfusionMatrix(ref, tagged)
print(cm)
sys.stdout.close()
sys.stdout=stdoutOrigin